# MCP Features: Resources, Prompts & Sampling

This notebook covers three MCP features that extend beyond basic tool calling:

1. **Resources 📄** - Exposing readable data/documents
2. **Prompts 💬** - Reusable AI workflow templates
3. **Sampling 🤖** - Bidirectional LLM communication (Advanced)



---

# Section 1: Resources 📄

## What Are Resources?

**Resources** are data or content that an MCP server exposes for reading. Unlike tools (which are functions that *do* things), resources are documents, files, or structured data that an LLM can *access*.

### Resources vs Tools - The Key Difference

| Concept | Type | Example |
|---------|------|----------|
| **Tools** | Verbs (actions) | `search_employees()`, `calculate_salary()`, `update_record()` |
| **Resources** | Nouns (data) | Employee handbook document, Remote work policy PDF, Org chart |

### Why Use Resources?

Instead of creating a tool for every piece of content:
- ❌ `get_employee_handbook()`, `get_remote_policy()`, `get_benefits_guide()` → 100s of tools
- ✅ Expose documents as **resources** with URIs → LLM discovers and reads them as needed

### URI Schemes

Resources are identified by **URIs** (similar to URLs):
```
hr://handbook/employee-benefits
hr://policy/remote-work
hr://job-description/senior-engineer
```

The server defines the URI scheme and structure.

## Demo: HR Document Server with Resources

Let's build a simple HR server that exposes company documents as resources.

In [ ]:
# Mock HR document data
HR_DOCUMENTS = {
    "hr://handbook/employee-benefits": {
        "title": "Employee Benefits Guide",
        "mimeType": "text/plain",
        "content": """
Employee Benefits Guide
=======================

Health Insurance:
- Medical: 100% premium covered for employees, 75% for dependents
- Dental: 100% premium covered for employees and dependents
- Vision: Available with $0 employee contribution

Retirement:
- 401(k) with 6% company match
- Immediate vesting

Time Off:
- 20 days PTO (Paid Time Off) per year
- 10 paid holidays
- Unlimited sick days
"""
    },
    "hr://policy/remote-work": {
        "title": "Remote Work Policy",
        "mimeType": "text/plain",
        "content": """
Remote Work Policy
==================

Eligibility:
- All full-time employees after 3 months of employment
- Manager approval required

Remote Work Options:
- Fully remote: Work from anywhere in the US
- Hybrid: Minimum 2 days/week in office
- Flexible: Choose your schedule with team coordination

Equipment:
- Company provides laptop, monitor, keyboard, mouse
- $500 annual home office stipend

Requirements:
- Reliable high-speed internet (minimum 25 Mbps)
- Dedicated workspace
- Available during core hours (10am-3pm local time)
"""
    },
    "hr://policy/pto-accrual": {
        "title": "PTO Accrual Policy",
        "mimeType": "text/plain",
        "content": """
PTO Accrual Policy
==================

Accrual Rate:
- Years 0-2: 15 days per year (1.25 days/month)
- Years 3-5: 20 days per year (1.67 days/month)
- Years 6+: 25 days per year (2.08 days/month)

Rollover:
- Maximum 40 days can be carried to next year
- Unused days beyond 40 are forfeited

Payout:
- Upon termination: Up to 20 days paid out
- Voluntary resignation: Must give 2 weeks notice for payout
"""
    },
    "hr://job-description/senior-engineer": {
        "title": "Senior Software Engineer - Job Description",
        "mimeType": "text/plain",
        "content": """
Senior Software Engineer
========================

Department: Engineering
Reports to: Engineering Manager
Location: Remote (US) or Hybrid (San Francisco, NYC)
Salary Range: $150,000 - $200,000

Responsibilities:
- Design and implement scalable backend services
- Mentor junior engineers and conduct code reviews
- Lead technical architecture discussions
- Collaborate with product and design teams
- Participate in on-call rotation

Requirements:
- 5+ years professional software development experience
- Strong proficiency in Python, Go, or Java
- Experience with distributed systems and microservices
- Excellent communication and collaboration skills
- Bachelor's degree in CS or equivalent experience

Nice to Have:
- Experience with Kubernetes and cloud platforms (AWS/GCP)
- Open source contributions
- Previous startup experience
"""
    },
    "hr://org-chart/engineering": {
        "title": "Engineering Organization Chart",
        "mimeType": "text/plain",
        "content": """
Engineering Organization
========================

VP of Engineering: Sarah Chen
├── Director of Backend: Michael Rodriguez
│   ├── Backend Team Lead: James Kim (5 engineers)
│   └── Infrastructure Team Lead: Lisa Zhang (4 engineers)
├── Director of Frontend: Emily Johnson
│   ├── Web Team Lead: David Park (6 engineers)
│   └── Mobile Team Lead: Anna Martinez (4 engineers)
└── Director of Platform: Robert Lee
    ├── DevOps Team Lead: Jennifer Taylor (3 engineers)
    └── Data Team Lead: Kevin Wilson (5 engineers)

Total Headcount: 32 engineers
"""
    },
    "hr://handbook/code-of-conduct": {
        "title": "Code of Conduct",
        "mimeType": "text/plain",
        "content": """
Code of Conduct
===============

Core Values:
1. Respect: Treat all colleagues with dignity and respect
2. Integrity: Act honestly and ethically in all business dealings
3. Collaboration: Work together toward common goals
4. Excellence: Strive for high quality in everything we do
5. Inclusion: Foster a diverse and welcoming workplace

Unacceptable Behavior:
- Harassment or discrimination of any kind
- Bullying or intimidation
- Sharing confidential company information
- Conflicts of interest without disclosure

Reporting:
- Report violations to HR at hr@company.com
- Anonymous hotline: 1-800-ETHICS-1
- All reports investigated confidentially
"""
    }
}

print(f"Loaded {len(HR_DOCUMENTS)} HR documents")

In [ ]:
# HR Resource Server implementation
class HRResourceServer:
    """
    MCP server that exposes HR documents as resources.

    Resources allow LLMs to discover and read documents without needing
    a separate tool for each document.
    """

    def __init__(self):
        self.documents = HR_DOCUMENTS

    def list_resources(self):
        """
        List all available resources.

        This is like a directory listing - the LLM can see what's available
        and decide what to read.

        Returns:
            List of resource descriptors with URIs and metadata
        """
        resources = []
        for uri, doc in self.documents.items():
            resources.append({
                "uri": uri,
                "name": doc["title"],
                "mimeType": doc["mimeType"],
                "description": f"HR document: {doc['title']}"
            })
        return resources

    def read_resource(self, uri):
        """
        Read the content of a specific resource.

        Args:
            uri: The unique identifier for the resource

        Returns:
            Resource content or error message
        """
        if uri not in self.documents:
            return {
                "error": f"Resource not found: {uri}",
                "available_resources": list(self.documents.keys())
            }

        doc = self.documents[uri]
        return {
            "uri": uri,
            "name": doc["title"],
            "mimeType": doc["mimeType"],
            "content": doc["content"]
        }

# Create server instance
hr_server = HRResourceServer()
print("✅ HR Resource Server initialized")

### Example 1: List All Available Resources

In [ ]:
# Discover what resources are available
resources = hr_server.list_resources()

print("Available HR Resources:")
print("=" * 70)
for resource in resources:
    print(f"📄 {resource['name']}")
    print(f"   URI: {resource['uri']}")
    print(f"   Type: {resource['mimeType']}")
    print()

### Example 2: Read a Specific Resource

In [ ]:
# Read the employee benefits guide
benefits_doc = hr_server.read_resource("hr://handbook/employee-benefits")

print(f"Document: {benefits_doc['name']}")
print("=" * 70)
print(benefits_doc['content'])

### Example 3: Read Remote Work Policy

In [ ]:
# Read the remote work policy
remote_policy = hr_server.read_resource("hr://policy/remote-work")

print(f"Document: {remote_policy['name']}")
print("=" * 70)
print(remote_policy['content'])

### Example 4: Error Handling - Invalid URI

In [ ]:
# Try to read a non-existent resource
result = hr_server.read_resource("hr://policy/nonexistent")

if "error" in result:
    print("❌ Error:", result['error'])
    print("\n💡 Available resources:")
    for uri in result['available_resources'][:3]:
        print(f"   - {uri}")
else:
    print(result['content'])

## 🎯 Exercise 1: Add Training Materials Resources

Your task: Extend the HR server to include **training materials** as resources.

**Requirements:**
1. Add 2 new training documents to `HR_DOCUMENTS`
2. Use the URI scheme: `hr://training/<topic>`
3. Each document should have `title`, `mimeType`, and `content`

**Suggested topics:**
- `hr://training/python-basics` - Python programming introduction
- `hr://training/security-awareness` - Security best practices

Fill in the blanks below:

In [ ]:
# Exercise 1: Add training materials

# Add these two resources to HR_DOCUMENTS dictionary
HR_DOCUMENTS["hr://training/python-basics"] = {
    "title": "______________________",  # TODO: Add title
    "mimeType": "text/plain",
    "content": """
# TODO: Write content for Python basics training (5-10 lines)
# Include: What is Python, basic syntax, why we use it




"""
}

HR_DOCUMENTS["hr://training/security-awareness"] = {
    "title": "______________________",  # TODO: Add title
    "mimeType": "text/plain",
    "content": """
# TODO: Write content for security training (5-10 lines)
# Include: Password best practices, phishing awareness, data protection




"""
}

# Test your implementation
hr_server = HRResourceServer()  # Recreate server with new documents
training_resources = [r for r in hr_server.list_resources() if "training" in r['uri']]

print("Training Resources Added:")
for resource in training_resources:
    print(f"✅ {resource['name']} ({resource['uri']})")

# Read one of your new resources
print("\n" + "="*70)
doc = hr_server.read_resource("hr://training/python-basics")
print(doc['content'])

## 🎓 Key Takeaways: Resources

✅ **Resources = Readable data/documents** (not executable functions)  
✅ **URIs identify resources** (like URLs for documents)  
✅ **Two operations**: `list_resources()` (discover) and `read_resource(uri)` (access)  
✅ **Better than individual tools** when you have many similar documents  
✅ **LLMs can discover** what's available and read on-demand

**When to use Resources vs Tools:**
- Use **Resources**: Static documents, reference materials, knowledge bases
- Use **Tools**: Dynamic operations, searches, calculations, updates

---

# Section 2: Prompts 💬

## What Are Prompts?

**Prompts** are reusable AI workflow templates that MCP servers can provide. Think of them as "recipes" or "checklists" for common tasks that ensure consistency and thoroughness.

### Why Not Just Hardcode Prompts?

You *could* hardcode prompts in your application, but MCP prompts offer:

1. **Discoverability** - LLMs can discover what workflows are available
2. **Dynamic Arguments** - Prompts accept parameters (employee_id, date range, etc.)
3. **Centralization** - Update the prompt once, everyone benefits
4. **Domain Expertise** - Encode expert knowledge into consistent workflows
5. **Versioning** - Evolve prompts over time without changing client code

### Real-World Use Case

**Without MCP Prompts:**
- HR managers conduct performance reviews inconsistently
- Some forget to check attendance, others skip goal-setting discussions
- Quality varies by manager

**With MCP Prompts:**
- Senior HR encodes the "perfect performance review" as a prompt
- All managers invoke: `conduct_performance_review(employee_id="EMP-123")`
- Consistent, thorough reviews every time

## Demo: HR Workflow Prompts

Let's build an HR server that provides prompt templates for common workflows.

In [ ]:
# Mock employee data for HR workflows
EMPLOYEES = {
    "EMP-001": {
        "name": "Alice Johnson",
        "role": "Senior Software Engineer",
        "department": "Engineering",
        "hire_date": "2021-03-15",
        "manager": "Sarah Chen",
        "performance_rating": "Exceeds Expectations",
        "projects": ["Payment API Redesign", "Database Migration"],
        "skills": ["Python", "PostgreSQL", "Kubernetes"]
    },
    "EMP-002": {
        "name": "Michael Rodriguez",
        "role": "Product Designer",
        "department": "Design",
        "hire_date": "2022-06-01",
        "manager": "Emily Johnson",
        "performance_rating": "Meets Expectations",
        "projects": ["Mobile App Redesign", "Design System v2"],
        "skills": ["Figma", "User Research", "Prototyping"]
    },
    "EMP-003": {
        "name": "Jennifer Park",
        "role": "Data Analyst",
        "department": "Analytics",
        "hire_date": "2023-01-10",
        "manager": "Kevin Wilson",
        "performance_rating": "Exceeds Expectations",
        "projects": ["Customer Churn Analysis", "Revenue Dashboard"],
        "skills": ["SQL", "Python", "Tableau", "Statistics"]
    },
    "EMP-004": {
        "name": "David Kim",
        "role": "Engineering Manager",
        "department": "Engineering",
        "hire_date": "2020-08-20",
        "manager": "Sarah Chen",
        "performance_rating": "Exceeds Expectations",
        "projects": ["Team Growth", "System Architecture"],
        "skills": ["Leadership", "System Design", "Python", "Go"]
    }
}

print(f"Loaded {len(EMPLOYEES)} employee records")

In [ ]:
# HR Prompt Server implementation
class HRPromptServer:
    """
    MCP server that provides reusable prompt templates for HR workflows.

    Prompts ensure consistent, thorough execution of common tasks.
    """

    def __init__(self):
        self.employees = EMPLOYEES

    def list_prompts(self):
        """
        List all available prompt templates.

        Returns:
            List of prompt descriptors with names, descriptions, and required arguments
        """
        return [
            {
                "name": "conduct_performance_review",
                "description": "Comprehensive performance review workflow for annual/quarterly reviews",
                "arguments": [
                    {"name": "employee_id", "description": "Employee ID (e.g., EMP-001)", "required": True},
                    {"name": "review_period", "description": "Review period (e.g., 'Q4 2024', '2024 Annual')", "required": True}
                ]
            },
            {
                "name": "generate_job_posting",
                "description": "Generate a detailed job posting with standard company format",
                "arguments": [
                    {"name": "role", "description": "Job title", "required": True},
                    {"name": "department", "description": "Department name", "required": True},
                    {"name": "level", "description": "Seniority level (Junior/Mid/Senior/Staff)", "required": True}
                ]
            },
            {
                "name": "onboard_new_employee",
                "description": "Complete onboarding checklist for new hires",
                "arguments": [
                    {"name": "employee_name", "description": "Full name of new employee", "required": True},
                    {"name": "role", "description": "Job title", "required": True},
                    {"name": "start_date", "description": "Start date (YYYY-MM-DD)", "required": True},
                    {"name": "manager", "description": "Manager's name", "required": True}
                ]
            }
        ]

    def get_prompt(self, name, arguments):
        """
        Generate a prompt from a template with provided arguments.

        Args:
            name: Name of the prompt template
            arguments: Dictionary of argument values

        Returns:
            Generated prompt text or error message
        """
        if name == "conduct_performance_review":
            return self._performance_review_prompt(arguments)
        elif name == "generate_job_posting":
            return self._job_posting_prompt(arguments)
        elif name == "onboard_new_employee":
            return self._onboarding_prompt(arguments)
        else:
            return f"Error: Unknown prompt '{name}'"

    def _performance_review_prompt(self, args):
        """Generate performance review prompt"""
        employee_id = args.get("employee_id")
        review_period = args.get("review_period")

        # Get employee data if available
        employee_context = ""
        if employee_id in self.employees:
            emp = self.employees[employee_id]
            employee_context = f"""
Employee Information:
- Name: {emp['name']}
- Role: {emp['role']}
- Department: {emp['department']}
- Manager: {emp['manager']}
- Hire Date: {emp['hire_date']}
- Recent Projects: {', '.join(emp['projects'])}
- Key Skills: {', '.join(emp['skills'])}
"""

        return f"""
Performance Review Workflow - {review_period}
{'='*60}

Employee: {employee_id}
{employee_context}

You are conducting a comprehensive performance review. Follow this structured approach:

## 1. Review Performance Data
- Analyze completed projects and deliverables from {review_period}
- Review any performance metrics (e.g., sprint velocity, code reviews, customer feedback)
- Check attendance and punctuality records

## 2. Assess Core Competencies
Evaluate the employee on these dimensions (scale: Needs Improvement / Meets Expectations / Exceeds Expectations):
- **Technical Skills**: Proficiency in role-specific skills
- **Communication**: Written and verbal communication effectiveness
- **Collaboration**: Teamwork and cross-functional partnership
- **Initiative**: Proactivity and problem-solving
- **Quality of Work**: Attention to detail and thoroughness

## 3. Highlight Achievements
- List 3-5 major accomplishments during {review_period}
- Identify specific examples where the employee exceeded expectations
- Note any positive feedback from peers or stakeholders

## 4. Identify Growth Areas
- List 2-3 areas where the employee can improve
- Frame constructively with specific, actionable feedback
- Suggest resources or training opportunities

## 5. Set Goals for Next Period
- Define 3-5 SMART goals (Specific, Measurable, Achievable, Relevant, Time-bound)
- Align goals with department objectives and career growth
- Discuss skill development opportunities

## 6. Overall Rating
Provide an overall performance rating:
- [ ] Needs Improvement
- [ ] Meets Expectations
- [ ] Exceeds Expectations
- [ ] Outstanding

## 7. Compensation Discussion
- Based on performance, is a raise or bonus warranted?
- If yes, recommend percentage/amount with justification

## 8. Next Steps
- Schedule follow-up check-in (30/60/90 days)
- Document any commitments made during review
- Update employee development plan

---
**Important**: Be specific, fair, and constructive. Balance praise with growth opportunities.
"""

    def _job_posting_prompt(self, args):
        """Generate job posting prompt"""
        role = args.get("role")
        department = args.get("department")
        level = args.get("level")

        return f"""
Job Posting Generator - {role}
{'='*60}

Create a compelling and detailed job posting for:
- **Role**: {role}
- **Department**: {department}
- **Level**: {level}

Your job posting should include the following sections:

## About the Role
Write 2-3 paragraphs describing:
- What this person will do day-to-day
- How this role contributes to the team and company mission
- What makes this role exciting and impactful

## Responsibilities
List 5-7 key responsibilities:
- Start each with an action verb (Lead, Build, Design, Analyze, etc.)
- Be specific about deliverables and outcomes
- Include both technical and collaborative aspects

## Requirements
**Must-Have Qualifications** (5-7 items):
- Years of experience appropriate for {level} level
- Technical skills and domain knowledge
- Educational background (if applicable)
- Soft skills (communication, collaboration, etc.)

**Nice-to-Have** (3-4 items):
- Additional skills that would be beneficial
- Industry experience
- Certifications or specializations

## What We Offer
Highlight company benefits:
- Competitive salary and equity
- Health, dental, vision insurance (100% covered)
- 401(k) with 6% match
- 20 days PTO + 10 holidays
- Remote-friendly / Hybrid options
- Professional development budget
- Parental leave

## About Us
Include a brief company overview:
- What we do and our mission
- Company culture and values
- Recent milestones or growth

## How to Apply
Include application instructions:
- Link to apply: careers@company.com
- What to include (resume, portfolio, cover letter)
- Expected timeline for interviews

---
**Tone**: Professional yet approachable. Focus on impact and growth opportunities.
**Length**: Aim for 500-700 words total.
"""

    def _onboarding_prompt(self, args):
        """Generate onboarding checklist prompt"""
        employee_name = args.get("employee_name")
        role = args.get("role")
        start_date = args.get("start_date")
        manager = args.get("manager")

        return f"""
New Employee Onboarding Checklist
{'='*60}

New Hire Information:
- **Name**: {employee_name}
- **Role**: {role}
- **Start Date**: {start_date}
- **Manager**: {manager}

Complete the following onboarding tasks to ensure a smooth first week:

## Pre-Day 1 (HR Tasks)
- [ ] Send welcome email with start date, time, and location
- [ ] Complete I-9 and tax forms
- [ ] Add to payroll system
- [ ] Order laptop and equipment (ship to home address if remote)
- [ ] Set up company email: {employee_name.lower().replace(' ', '.')}@company.com
- [ ] Create accounts: Slack, GitHub, Zoom, Google Workspace
- [ ] Add to relevant Slack channels and email lists
- [ ] Assign onboarding buddy

## Day 1 - Welcome & Orientation
- [ ] Welcome meeting with manager ({manager})
- [ ] HR orientation (benefits, policies, culture)
- [ ] IT setup: laptop, accounts, VPN, tools
- [ ] Office tour (if on-site) or virtual team introductions
- [ ] Assign first task or reading materials
- [ ] Schedule 1:1s with key team members

## Week 1 - Getting Started
- [ ] Complete required training: Security Awareness, Code of Conduct
- [ ] Read employee handbook and key documentation
- [ ] Set up development environment (for engineers)
- [ ] Shadow team members to understand workflows
- [ ] Attend team meetings and standups
- [ ] Complete first small task or project
- [ ] Daily check-ins with manager

## Week 2-4 - Ramp Up
- [ ] Complete role-specific training modules
- [ ] Start contributing to team projects
- [ ] Schedule 1:1s with cross-functional partners
- [ ] Review team goals and OKRs
- [ ] Set 30-60-90 day goals with manager
- [ ] Provide onboarding feedback to HR

## 30-Day Check-In
Schedule a formal check-in to discuss:
- How is onboarding going?
- Any blockers or concerns?
- Progress toward initial goals
- Clarify expectations for the next 60 days

## Resources to Share
- Employee handbook: hr://handbook/employee-benefits
- Remote work policy: hr://policy/remote-work
- Code of conduct: hr://handbook/code-of-conduct
- Department org chart
- Project documentation and wikis

---
**Goal**: Make {employee_name} feel welcome, set up for success, and productive by end of Week 2.
"""

# Create server instance
hr_prompt_server = HRPromptServer()
print("✅ HR Prompt Server initialized")

### Example 1: List Available Prompts

In [ ]:
# Discover what prompt workflows are available
prompts = hr_prompt_server.list_prompts()

print("Available HR Workflow Prompts:")
print("=" * 70)
for prompt in prompts:
    print(f"💬 {prompt['name']}")
    print(f"   Description: {prompt['description']}")
    print(f"   Arguments:")
    for arg in prompt['arguments']:
        required = "(required)" if arg['required'] else "(optional)"
        print(f"      - {arg['name']}: {arg['description']} {required}")
    print()

### Example 2: Generate Performance Review Workflow

In [ ]:
# Generate a performance review prompt for a specific employee
review_prompt = hr_prompt_server.get_prompt(
    "conduct_performance_review",
    {
        "employee_id": "EMP-001",
        "review_period": "Q4 2024"
    }
)

print(review_prompt)

### Example 3: Generate Job Posting

In [ ]:
# Generate a job posting prompt
job_posting_prompt = hr_prompt_server.get_prompt(
    "generate_job_posting",
    {
        "role": "Senior Data Scientist",
        "department": "Data & Analytics",
        "level": "Senior"
    }
)

print(job_posting_prompt)

### Example 4: Generate Onboarding Checklist

In [ ]:
# Generate onboarding checklist for a new hire
onboarding_prompt = hr_prompt_server.get_prompt(
    "onboard_new_employee",
    {
        "employee_name": "Sarah Williams",
        "role": "Product Manager",
        "start_date": "2025-01-15",
        "manager": "Emily Johnson"
    }
)

print(onboarding_prompt)

## 🎯 Exercise 2: Create an Interview Planning Prompt

Your task: Add a new prompt template for **planning candidate interviews**.

**Requirements:**
1. Add the prompt definition to `list_prompts()`
2. Implement the prompt generator method
3. The prompt should help HR plan a structured interview process

**Prompt name**: `plan_interview`

**Arguments**:
- `candidate_name` (required) - Name of the candidate
- `role` (required) - Position they're interviewing for
- `interview_date` (required) - Scheduled interview date

Fill in the blanks below:

In [ ]:
# Exercise 2: Add interview planning prompt

# Step 1: Add this to the list_prompts() method
# (For the exercise, we'll modify it here, but in real code you'd edit the class)

interview_prompt_definition = {
    "name": "plan_interview",
    "description": "______________________",  # TODO: Add description
    "arguments": [
        # TODO: Add three argument definitions (candidate_name, role, interview_date)
        {"name": "______", "description": "______", "required": True},
        {"name": "______", "description": "______", "required": True},
        {"name": "______", "description": "______", "required": True},
    ]
}

print("✅ Prompt definition created:")
print(interview_prompt_definition)

In [ ]:
# Step 2: Implement the prompt generator

def generate_interview_prompt(candidate_name, role, interview_date):
    """
    Generate an interview planning prompt.

    The prompt should help HR plan a structured interview that covers:
    - Interview format (phone screen, technical, behavioral, etc.)
    - Key competencies to assess
    - Sample questions for each interview stage
    - Evaluation criteria
    - Next steps
    """

    return f"""
Interview Planning Guide
{'='*60}

Candidate: {candidate_name}
Role: {role}
Interview Date: {interview_date}

# TODO: Complete this interview planning prompt (15-25 lines)
# Include sections for:
# 1. Interview Format (rounds, durations, interviewers)
# 2. Key Competencies to Assess (technical skills, culture fit, etc.)
# 3. Sample Questions (5-7 questions across different categories)
# 4. Evaluation Criteria (how to score the candidate)
# 5. Next Steps (debrief, decision timeline)















"""

# Test your implementation
test_prompt = generate_interview_prompt(
    candidate_name="John Smith",
    role="Senior Backend Engineer",
    interview_date="2025-01-20"
)

print(test_prompt)

## 🎓 Key Takeaways: Prompts

✅ **Prompts = Reusable AI workflow templates** with dynamic arguments  
✅ **Ensure consistency** across users and over time  
✅ **Discoverable** - LLMs can see what workflows exist  
✅ **Centralized** - Update once, everyone benefits  
✅ **Encode expertise** - Senior knowledge → accessible workflows

**When to use Prompts:**
- Repetitive workflows that need consistency (reviews, reports, checklists)
- Complex multi-step processes that users might forget
- Domain expertise you want to standardize across the organization
- Situations where quality/thoroughness matters

---

# Section 3: Sampling 🤖 ⚠️ ADVANCED

## What Is Sampling?

**Sampling** is a bidirectional communication pattern where an **MCP server can ask the host's LLM for help**.

### Role Reversal

Normal flow:
```
Host LLM → calls tool on → MCP Server
```

With sampling:
```
Host LLM → calls tool on → MCP Server
                              ↓
                    (server needs AI help)
                              ↓
Host LLM ← sampling request ← MCP Server
    ↓
(generates text)
    ↓
Host LLM → sampling response → MCP Server
                              ↓
                    (returns final result)
```

### Why Is This Useful?

**Use cases:**

1. **Data transformation** - Server has raw data, needs it summarized/formatted
2. **AI-powered analysis** - Server needs LLM judgment (classify, extract, score)
3. **Privacy-preserving AI** - Process sensitive data locally, don't send raw data to external LLM
4. **Cost optimization** - Server pre-processes data, only sends relevant parts to LLM

### Real-World Example: E-commerce Payment Analysis

**Scenario**: You have customer payment history (sensitive financial data)

**Without sampling**:
```
Host LLM → "Get payment history" → Server
Server → returns 1000 transactions → Host LLM
Host LLM analyzes all transactions (slow, expensive, privacy risk)
```

**With sampling**:
```
Host LLM → "Analyze payment patterns" → Server
Server has 1000 transactions locally
Server → "Summarize these patterns" → Host LLM (sampling)
Host LLM → returns summary → Server
Server → returns summary + metadata → Host LLM
```

**Benefits**:
- ✅ Raw transaction data never leaves the server
- ✅ Only anonymized summaries shared
- ✅ Faster (less data transfer)
- ✅ Cheaper (fewer tokens to process)

## Sampling adds significant complexity ⚠️


1. **Conceptual complexity** - "Wait, the server calls the LLM back?"
2. **Bidirectional communication** - Requires proper MCP protocol implementation
3. **Latency** - Server → Host → LLM → Host → Server (multiple round trips)
4. **Cost considerations** - Each sampling request costs API tokens
5. **Permission model** - Host must allow servers to use sampling
6. **Can't run in Colab** - Requires subprocess communication

### When NOT to Use Sampling

❌ Don't use sampling if:
- You can just return the data and let the host LLM process it
- The data isn't sensitive (no privacy concerns)
- You're not trying to optimize token usage
- The server doesn't need AI-powered transformations

✅ Use sampling when:
- Server has sensitive data that shouldn't leave the system
- You want to pre-process large datasets before sending to LLM
- Server needs AI judgment but wants to control the context

## Sampling Architecture Diagram

```
┌─────────────────────────────────────────────────────────┐
│                     Host Application                    │
│                  (interactive_client.py)                │
│                                                         │
│  ┌────────────────────────────────────────────────┐     │
│  │           OpenAI gpt-5-nano                    │     │
│  │  "Analyze customer payment patterns"           │     │
│  └────────────────────────────────────────────────┘     │
│                          │                              │
│                          ▼                              │
│  ┌────────────────────────────────────────────────┐     │
│  │         MCP Orchestrator                       │     │
│  │  • Routes tool call to payment server          │     │
│  │  • Handles sampling requests from server       │     │
│  └────────────────────────────────────────────────┘     │
└────────────┬───────────────────────────────────────────-┘
             │ stdio transport
             ▼
┌─────────────────────────────────────────────────────────┐
│              Payment Analysis MCP Server                │
│           (sampling_demo.py - separate file)            │
│                                                         │
│  1. Receives: analyze_payment_patterns(customer_id)     │
│  2. Loads: 1000 transactions from local database        │
│  3. Sampling request → Host LLM:                        │
│     "Summarize spending patterns:                       │
│      [transaction data]"                                │
│  4. Receives summary from Host LLM                      │
│  5. Returns: {summary, transaction_count, metadata}     │
│                                                         │
│  🔒 Raw transaction data never sent to external A I     │
└─────────────────────────────────────────────────────────┘
```

## Implementation: See `sampling_demo.py`

Because sampling requires:
- Bidirectional stdio communication
- Subprocess management
- MCP protocol implementation

It **cannot run in Jupyter/Colab**.

A complete working implementation is provided in **`sampling_demo.py`**.

### What's in `sampling_demo.py`:

1. **E-commerce Payment Server** - MCP server with customer transaction data
2. **`analyze_payment_patterns` tool** - Uses sampling to summarize spending behavior
3. **Mock transaction data** - Realistic customer purchase history
4. **Sampling implementation** - Shows how server requests LLM help

### How to run it:

```bash
# Set your OpenAI API key
export OPENAI_API_KEY="sk-your-key-here"

# Run the sampling demo
python sampling_demo.py
```

### What you'll see:

```
Starting Payment Analysis Server with Sampling...

User Query: "Analyze spending patterns for customer CUST-001"
  
  🔧 Calling tool: analyze_payment_patterns
  📊 Server has 156 transactions for CUST-001
  🤖 Server requesting sampling from host LLM...
  💬 Sampling prompt: "Summarize spending behavior..."
  ✅ Sampling complete (returned summary)
  📤 Returning result to host

Final Response:
"Customer CUST-001 shows consistent monthly spending averaging $450.
 Primary categories: Electronics (40%), Clothing (30%), Books (20%).
 No late payments in the past 12 months. High loyalty score."
```

## 🎓 Key Takeaways: Sampling

✅ **Sampling = Server asks host LLM for help** (role reversal)  
✅ **Use case: Privacy-preserving AI** - Process sensitive data locally  
✅ **Advanced pattern** - Requires bidirectional MCP communication  
✅ **Cannot run in Colab** - Needs subprocess/stdio transport  
✅ **See `sampling_demo.py`** for full working implementation

**When to use Sampling:**
- Server has sensitive data that shouldn't leave the system
- Want to pre-process large datasets before sending to LLM
- Need AI-powered transformations but want to control context
- Cost optimization (reduce tokens sent to external API)

**Trade-offs:**
- ⚠️ Added latency (multiple round trips)
- ⚠️ Increased complexity
- ⚠️ Host must grant sampling permissions
- ⚠️ Each sampling request costs tokens

---

# Summary: Resources, Prompts & Sampling

## Feature Comparison

| Feature | What It Is | When to Use | Complexity | Runs in Colab? |
|---------|-----------|-------------|------------|----------------|
| **Resources 📄** | Readable data/documents | Static content, reference materials | ⭐ Easy | ✅ Yes |
| **Prompts 💬** | Reusable AI workflow templates | Consistent, repeatable tasks | ⭐⭐ Medium | ✅ Yes |
| **Sampling 🤖** | Server calls host LLM | Privacy-preserving AI, data pre-processing | ⭐⭐⭐ Advanced | ❌ No |

## Choosing the Right Feature

**I have documents/data to expose:**
→ Use **Resources** 📄

**I want consistent workflows/checklists:**
→ Use **Prompts** 💬

**I need to process sensitive data with AI:**
→ Use **Sampling** 🤖 (advanced)

**I want to do dynamic operations:**
→ Use **Tools** 🔧 (from basic tutorial)

---

## Additional Resources

- [MCP Official Documentation](https://github.com/anthropics/mcp)
- [MCP Python SDK](https://github.com/anthropics/python-sdk)

